<a href="https://colab.research.google.com/github/payalahuja98/SynMaxShipping/blob/main/Voyages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import pandas as pd

ports = pd.read_csv("./ports.csv", sep='\t')
tracking = pd.read_csv("./tracking.csv", sep="\t")

In [149]:
tracking = tracking.loc[tracking['speed'] == 0]
tracking['datetime'] = pd.to_datetime(tracking['datetime'], infer_datetime_format=True)
tracking = tracking.sort_values(by="datetime")
tracking

,vessel,datetime,lat,long,heading,speed,draft
51666,129,2019-01-01 00:08:00,-20.587000,116.763800,331.0,0.0,9.0
51672,129,2019-01-01 00:08:00,-20.587000,116.763800,331.0,0.0,9.0
51673,129,2019-01-01 00:08:00,-20.587000,116.763800,331.0,0.0,9.0
51659,129,2019-01-01 00:08:00,-20.587000,116.763800,331.0,0.0,9.0
51671,129,2019-01-01 00:08:00,-20.587000,116.763800,331.0,0.0,9.0
...,...,...,...,...,...,...,...
630900,85,2019-12-31 21:37:00,29.743603,-93.871295,226.0,0.0,9.6
742448,23,2019-12-31 21:39:00,36.999720,126.780920,129.0,0.0,11.1
630757,85,2019-12-31 21:52:00,29.743590,-93.871295,226.0,0.0,9.6
742435,23,2019-12-31 21:54:00,36.999710,126.780920,129.0,0.0,11.1


In [191]:
tol = 0.02
vessel = 0
date = tracking['datetime']
port = 0
output = pd.DataFrame(columns=['vessel','datetime','port'])

for i, rowV in tracking.iterrows(): 
  vessel = rowV['vessel']
  lat = rowV['lat']
  long = rowV['long']
  date = rowV['datetime']
  for j, rowP in ports.iterrows():
    if abs(long - rowP['long']) + abs(lat - rowP['lat']) <= tol:
      port = rowP['port']
      data=[vessel, date, port]
      output.loc[i] = (data)
      break

In [196]:
output = output.sort_values(by=['vessel','datetime'])

In [197]:
output.to_csv("output.csv", index=False)


In [230]:
voyages = pd.DataFrame(columns=['vessel', 'begin_date', 'end_date', 'begin_port_id', 'end_port_id'])
output = output.reset_index(drop=True)

vessel = output['vessel'][0]
begin_date = output['datetime'][0]
begin_port_id = output['port'][0]

for i, rowO in output.iterrows():
  if rowO['port'] != begin_port_id:
    if rowO['vessel'] != vessel:
      vessel = rowO['vessel']

    end_date = rowO['datetime']
    end_port_id = rowO['port']
    voyages.loc[i] = [vessel, begin_date, end_date, begin_port_id, end_port_id]
    begin_date = end_date
    begin_port_id = end_port_id
  

In [231]:
voyages.to_csv("voyages.csv", index=False)